In [20]:
from difflib import SequenceMatcher
import pandas as pd
import re
import os 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
combo = pd.read_csv('fs/combined.csv')
['Title', 'Url', 'Standfirst', 'Date']
# print(combo.columns.tolist())

['Title', 'Url', 'Standfirst', 'Date']

In [22]:
def search_similar(df,
                   queries: list[str],
                   columns: list[str],
                   threshold: float = 0.6,
                   token_overlap_min: float = 0.5):
    queries = [q.strip().casefold() for q in queries if q and str(q).strip()]
    if not queries:
        return df.iloc[0:0]

    word_re = re.compile(r"\w+")

    def token_overlap(a: str, b: str) -> float:
        sa, sb = set(word_re.findall(a)), set(word_re.findall(b))
        if not sa or not sb:
            return 0.0
        denom = min(len(sa), len(sb))
        return len(sa & sb) / denom

    def is_similar(text) -> bool:
        if pd.isna(text):
            return False
        t = str(text).casefold().strip()
        if not t:
            return False

        for q in queries:
            if q in t or t in q:
                return True
            if token_overlap(t, q) >= token_overlap_min:
                return True
            if SequenceMatcher(None, t, q).ratio() >= threshold:
                return True
        return False

    mask = df[columns].applymap(is_similar).any(axis=1)
    # return df[mask]
    returno = df[mask].copy()
    returno['Date'] = pd.to_datetime(returno['Date']).dt.strftime("%-I%p %d %b")
    # returno = returno[['title', 'link']]

    for index, row in returno.iterrows():
        print(row['Title'])
        print(row['Date'], row['Url'])
        print("\n")

In [23]:


####
terms = ["Environmental protection"]
###

search_similar(combo, terms, ['Title',  'Standfirst'], threshold=0.7)



Who’s responsible for our plastic problem? – Full Story podcastClimate and environment reporter Petra Stock tells Nour Haydar about Australia’s rising plastic waste problem and what needs to be done to tackle one of the biggest environmental threats of our timePodcast19:35
12AM 15 Sep https://www.theguardian.com/australia-news/audio/2025/sep/15/whos-responsible-for-our-plastic-problem-full-story-podcast


Can you trust your sunscreen? - Full Story podcastConsumer affairs reporter Catie Mcleod tells Nour Haydar why questions are being asked about the level of protection provided by some leading brands and what consumers need to know nowPodcast17:24
12AM 24 Jun https://www.theguardian.com/australia-news/audio/2025/jun/24/can-you-trust-your-sunscreen-full-story-podcast


Why is Formula One having a moment? – Full Story podcastMatilda Boseley talks to sport reporter Jack Snape on why despite the many controversies including sponsorship and environmental impact, the F1 is riding high with a